In [1]:
import pandas as pd
import numpy as np
import qiime2 as q2
import biom
from qiime2 import Artifact, Metadata, Visualization
from qiime2.plugins.emperor.actions import biplot
from qiime2.plugins.qurro.actions import loading_plot
from qiime2.plugins.gemelli.actions import (joint_rpca, filter_ordination, feature_correlation_table)
from qiime2.plugins.longitudinal.actions import volatility
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.feature_table.methods import filter_samples
from biom import load_table
# from gemelli.rpca import rpca
from qiime2.plugins.gemelli.actions import rpca

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import os

plt.style.use('ggplot')
%matplotlib inline

In [2]:
# load in biplot, biomtable, taxonomy, and metadata
table = Artifact.import_data("FeatureTable[Frequency]", "data/wolr2/gg2/zebra30/196551_zebra.biom/feature-table.biom")
metadata = Metadata.load('data/metadata/Metadata_and_Cumulative_TP.tsv')
taxonomy = Artifact.import_data('FeatureData[Taxonomy]', 'ref/lineages.txt')


In [3]:
# perform RPCA
# ordination, distance = rpca(table, min_sample_count=500, max_iterations=15)
ordination, distance = rpca(table, min_sample_count=500)


In [8]:
if not os.path.exists(f'data/wolr2/gg2/zebra30/rpca'):
    os.makedirs(f'data/wolr2/gg2/zebra30/rpca')

biplot_viz = biplot(ordination, metadata, ignore_missing_samples=True, number_of_features=50)
biplot_viz.visualization.save('data/wolr2/gg2/zebra30/rpca/emp_biplot_50.qzv')

'data/wolr2/gg2/zebra30/rpca/emp_biplot_50.qzv'

In [9]:
type(biplot_viz)

qiime2.sdk.results.Results

In [12]:
# Ensure the output directory exists
if not os.path.exists(f'data/wolr2/gg2/zebra30/qurro'):
    os.makedirs(f'data/wolr2/gg2/zebra30/qurro')


# run Qurro metaG
qurro_plot_metag = loading_plot(ordination, table, metadata, feature_metadata=taxonomy.view(Metadata))
qurro_plot_metag.visualization.save(f'data/wolr2/gg2/zebra30/qurro/qurro-metag.qzv')

'data/wolr2/gg2/zebra30/qurro/qurro-metag.qzv'

### pulling out log ratios(up & down regulated)